## Fireship GPT
an attempt at making an LLM emulate the tone, pacing, and content style of [Fireship](https://www.youtube.com/@Fireship) by fine-tuning them on curated datasets.

In [ ]:
import torch
from transformers import GenerationConfig, TextIteratorStreamer, TextStreamer , AutoModelForCausalLM , AutoTokenizer

model = AutoModelForCausalLM .from_pretrained('AdithyaSK/Fireship-GPT-v1')
tokenizer = AutoTokenizer.from_pretrained('AdithyaSK/Fireship-GPT-v1')
model = model.to("cuda")

In [ ]:
# input the title of the video
video_title = "Rust in 100 seconds"

# input the a small summary of the video
video_summary = ""


prompt = f"""
[INST]
You are youtuber called Fireship you make engaging high-intensity and entertaining coding tutorials and tech news. 
you covers a wide range of topics relevant to programmers, aiming to help them learn and improve their skills quickly.

Given the title of the video : {video_title} 
and a small summary : {video_summary}
[/INST]

Generate the video : 
"""

generation_config = GenerationConfig(
    repetition_penalty=1.1,
    max_new_tokens=1024,
    temperature=0.9,
    top_p=0.95,
    top_k=40,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    do_sample=True,
    use_cache=True,
    return_dict_in_generate=True,
    output_attentions=False,
    output_hidden_states=False,
    output_scores=False,
)
streamer = TextStreamer(tokenizer)
batch = tokenizer(str(prompt.strip()), return_tensors="pt", add_special_tokens=True)
generated = model.generate(
    inputs=batch["input_ids"].to("cuda"),
    generation_config=generation_config,
    streamer=streamer,
)
print(tokenizer.decode(generated["sequences"].cpu().tolist()[0]))